<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing_this_time_it_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization, \
                             GlobalAveragePooling2D, Concatenate, Reshape, Input, UpSampling2D

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteENCIOD/cifar100-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteENCIOD/cifar100-test.tfrecord*...:   0%|         …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [ ]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

### Dataset to numpy:

### Preprocessing #2

In [ ]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

In [119]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16
NUM_CLASSES = ds_info.features['label'].num_classes

ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTO)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [225]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_VGG

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_Resnet50

from tensorflow.keras.applications.densenet import DenseNet121

In [257]:
tf.keras.backend.clear_session()

In [258]:
# Load the VGG16 model with ImageNet weights
vgg16 = VGG16(weights='imagenet', include_top=False)

# Load the DenseNet121 model with ImageNet weights
densenet121 = DenseNet121(weights='imagenet', include_top=False)

vgg16.trainable = False
densenet121.trainable = False
# Reshape the output of VGG16 to match the input shape of DenseNet121
# Assuming a common denominator shape for both
vgg16_output = vgg16.output
reshape_layer = Reshape((32, 32, 3))(vgg16_output)

# Add the reshaped output of VGG16 as input to DenseNet121
densenet121_input = densenet121(reshape_layer)

In [ ]:
# Add a global average pooling layer to reduce the dimensions
gap_layer = GlobalAveragePooling2D()(densenet121_input)

# Add a fully connected layer with, for example, 256 neurons
fc_layer = Dense(256, activation='relu')(gap_layer)

# Add a final output layer, for example, for a classification task with 10 classes
output_layer = Dense(NUM_CLASSES, activation='softmax')(fc_layer)

# Create a new model with the input of VGG16 and the new output layer
custom_model = Model(inputs=vgg16.input, outputs=output_layer)

In [ ]:
# Display the summary of the new model
custom_model.summary(show_trainable = True)

In [261]:
custom_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
custom_model.fit(ds_train, epochs = 2)

In [263]:
ds_train.element_spec

(TensorSpec(shape=(None, None, 32, 32, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(None, None), dtype=tf.int64, name=None))